Before you get started you're gonna have to do something like:

```
cp -r  ~/Library/Application\ Support/MobileSync/Backup/44df4b8f8399beb6b4e374c33b0ff75e603c06c9/3d/3d0d7e5fb2ce288813306e4d4636395e047a3d28 ~/work/notebooks/chat.db

cp -r  ~/Library/Application\ Support/MobileSync/Backup/44df4b8f8399beb6b4e374c33b0ff75e603c06c9/31/31bb7ba8914766d4ba40d6dfb6113c8b614be442 ~/work/notebooks/contacts.db
```

The filepaths above are for Mac. **And you need to do an unencrypted iTunes backup of your iPhone.**

In [ ]:
import sqlite3
import pandas as pd
# watch out, i pip installed this in a docker container so we'll see how this goes
from phonenumbers import format_number, parse as pn_parse
from phonenumbers import PhoneNumberFormat
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def parse_phone(x):
    try:
        return format_number(pn_parse(x, 'US'), PhoneNumberFormat.E164)
    except:
        return x
    
def explore_db(conn):
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cur.fetchall())

## The Chat DB

In [ ]:
conn = sqlite3.connect('chat.db')
explore_db(conn)

In [ ]:
query = """
SELECT chat.chat_identifier, message.is_from_me, 
datetime(message.date/1000000000 + strftime('%s', '2001-01-01 00:00:00'),
       'unixepoch', 'localtime') as date, 
message.text 
FROM chat 
JOIN chat_message_join on chat.ROWID = chat_message_join.chat_id
JOIN message on message.ROWID = chat_message_join.message_id
"""

In [ ]:
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
df_canonicalized_phone = pd.read_sql('SELECT * FROM handle', conn)

## The Contacts DB

In [ ]:
conn = sqlite3.connect('contacts.db')
explore_db(conn)

In [ ]:
query = """
SELECT First, Last, Value 
FROM ABPerson 
LEFT OUTER JOIN ABMultiValue 
ON ABPerson.ROWID = ABMultiValue.record_id
"""

In [ ]:
df_names = pd.read_sql(query, conn)
df_names = df_names.fillna("")
df_names['First_Last'] = df_names[['First', 'Last']].apply(lambda x: '_'.join(x), axis=1)
df_names['parsed_phone'] = df_names.value.apply(parse_phone)
df_names = df_names[df_names.parsed_phone != ""]

## Creating the final dataset

In [ ]:
df = df.merge(df_names, left_on='chat_identifier', right_on='parsed_phone', how='left')

In [ ]:
conn = sqlite3.connect('analysis.db')
df.to_sql("messages", conn, index=False, if_exists='replace')